In [2]:
import numpy as np
import pandas as pd

import emcee
import corner

import os
CWD = "/".join(os.getcwd().split("/")[:-2])

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ID = 100004076#237880881

In [7]:
props = pd.read_csv(f"{CWD}/data/summary/everything.csv")
#target = props[props.ID == ID]
#target
props.ID

0    100004076
1    212035340
2    237880881
3     44984200
4    277539431
5    300741820
Name: ID, dtype: int64

In [0]:
lc = pd.read_csv(f"{CWD}/data/lcs")

In [0]:
# see here https://emcee.readthedocs.io/en/stable/tutorials/monitor/
filename = f"{CWD}/analysis/results/mcmc/{target.ID}_MCMC.h5"
reader = emcee.backends.HDFBackend(filename)

tau = reader.get_autocorr_time()
burnin = int(2 * np.max(tau))
thin = int(0.5 * np.min(tau))
samples = reader.get_chain(discard=burnin, flat=True, thin=thin)
log_prob_samples = reader.get_log_prob(discard=burnin, flat=True, thin=thin)
log_prior_samples = reader.get_blobs(discard=burnin, flat=True, thin=thin)

print("burn-in: {0}".format(burnin))
print("thin: {0}".format(thin))
print("flat chain shape: {0}".format(samples.shape))
print("flat log prob shape: {0}".format(log_prob_samples.shape))
print("flat log prior shape: {0}".format(log_prior_samples.shape))

# Plots
### Plot the full chain

In [0]:
fig, axes = plt.subplots(6, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ['flare peak\nlongitude', 'flare\nlatitude', 'relative\namplitude', 'flare\nFWHM', 'inclination \nrad]', 'phi0\n[rad]']
for j in range(ndim):
    ax = axes[j]
    ax.plot(samples[:, :, j], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[j])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");
plt.savefig(f"{CWD}/analysis/plots/flarefit/{target.prefix}{target.ID}_chain.png",dpi=300)

### Corner plot

In [0]:

flat_samples = sampler.get_chain(discard=burnin, flat=True)
print(flat_samples.shape)
fig = corner.corner(flat_samples, labels=labels)# truths=[phi_a, theta_a, a, fwhm,i_mu, phi0]);
plt.savefig(f"{CWD}/analysis/plots/flarefit/{target.prefix}{target.ID}_corner_disc{burnin}.png",dpi=300)

### Retrieve and plot some example fits to the data from the chain

In [0]:
N = 500
inds = np.random.randint(len(flat_samples), size=N)
plt.figure(figsize=(10,8))
#plt.scatter(flck.time, flck.flux, s=1)
plt.scatter(phi, flux, s=10, c="k")
#plt.errorbar(phi, flux, yerr=flux_err, fmt=".k", capsize=0)

for ind in inds:
    sample = flat_samples[ind]
    phi_a, theta_a, a, fwhm, i, phi0 = sample
   # plt.plot(phi, aflare(phi, phi_a, fwhm, a*median)+ median, c="grey", alpha=0.05, linewidth=3)
    plt.plot(phi, big_model(phi_a, theta_a, a, fwhm, i, phi0=phi0,
               phi=phi, num_pts=50, qlum=qlum,
               Fth=Fth, R=R, median=median), c="r", alpha=0.5,)
#plt.plot(phi, flare+median, c="r",linewidth=1)
plt.legend(fontsize=14)
plt.xlabel("time")
plt.ylabel("flux");
plt.savefig(f"{CWD}/analysis/plots/flarefit/{target.prefix}{target.ID}_flarefit_{N}retrievals.png",dpi=300)

# Results
### Percentiles of fitted parameters

In [0]:
percentiles = [16,50,84]

In [0]:
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
#map phi_a_distr to t0_distr:
#samples[:, 0] = np.exp(samples[:, 2])
# map phi0 to phi_peak longitude, still call it phi0
#samples[:, -1] = samples[:, 0]%(2.*np.pi) - samples[:, -1] 3 0 would be facing the observer
t0_p, theta_a_p, a_p, fwhm_p, i_p, phi0_p = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                                   zip(*np.percentile(samples, perentiles,
                                                       axis=0)))


In [0]:
# Write percentiles to file
with open(f"{CWD}/analysis/results/mcmc/mcmcoutput.csv","a") as f:
    firstout = ("ID,tpeak{percentiles[0]},tpeak{percentiles[1]},tpeak{percentiles[2]}"/
                "theta_peak{percentiles[0]},theta_peak{percentiles[1]},theta_peak{percentiles[2]}")
    #Add more lines here
    out = f"{target.ID},{*t0_p},{*theta_a_p}\n"
    f.write(out)

### Equivalent duration distribution

In [0]:
def calculate_ED(t, t0, dur, ampl, median):
    x = t * 60.0 * 60.0 * 24.0
    return np.sum(np.diff(x) * aflare(t, t0, dur, ampl)[:-1]) / (dur * 6 * median)

In [0]:

#Find t_peak_distr from phia_distr

In [0]:
t0_distr = samples[:,0]

a_distr = samples[:,2]
fwhm_distr = samples[:,3]

ED_distr = calculate_ED(lc.t.values, t0_distr, fwhm_distr, a_distr )